### Add HydroBasin data to Postgis Database server

* Purpose of script: Ingest Data from HydroBasins last step into a postGIS database
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20171110

The script requires a file called .password to be stored in the current working directory with the password to the database.

Please note that columns with uppercase should be referred to by using double quotes whereas strings need single quotes. 

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M11D16 UTC 11:21


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
SCRIPT_NAME = "Y2017M11D15_RH_Add_HydroBasins_postGIS_V01"

INPUT_VERSION = 3

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

INPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V%0.2d" %(INPUT_VERSION)

# Database settings
DATABASE_IDENTIFIER = "aqueduct30v01"
DATABASE_NAME = "database01"
TABLE_NAME = "hybasvalid04"

In [3]:
import os
import boto3
import botocore
from sqlalchemy import *
import geopandas as gpd
import pandas as pd
from shapely.geometry.multipolygon import MultiPolygon
from geoalchemy2 import Geometry, WKTElement

In [46]:
%matplotlib inline

In [4]:
rds = boto3.client('rds')

In [5]:
F = open(".password","r")
password = F.read().splitlines()[0]
F.close()

In [6]:
response = rds.describe_db_instances(DBInstanceIdentifier="%s"%(DATABASE_IDENTIFIER))

In [7]:
status = response["DBInstances"][0]["DBInstanceStatus"]

In [8]:
print(status)

available


In [9]:
endpoint = response["DBInstances"][0]["Endpoint"]["Address"]

In [10]:
print(endpoint)

aqueduct30v01.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com


In [11]:
engine = create_engine('postgresql://rutgerhofste:%s@%s:5432/%s' %(password,endpoint,DATABASE_NAME))

In [39]:
connection = engine.connect()

In [13]:
!rm -r {EC2_INPUT_PATH}
!rm -r {EC2_OUTPUT_PATH}

!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [14]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive --quiet

In [15]:
gdf = gpd.read_file(os.path.join(EC2_INPUT_PATH,INPUT_FILENAME+".shp"))

In [16]:
gdf = gdf.set_index("PFAF_ID", drop=False)

In [17]:
gdf.columns = map(str.lower, gdf.columns)

for PostgreSQL its better to have non-duplicate tables whereas for Pandas having duplicate column names is better. Renaming.  

In [18]:
gdf.columns = ['pfaf_id2', 'geometry']

In [19]:
gdf.head()

,pfaf_id2,geometry
PFAF_ID,,
611001,611001,(POLYGON ((-78.99722222222219 9.45416666666669...
611002,611002,POLYGON ((-77.00416666666663 5.770833333333362...
611003,611003,POLYGON ((-76.88749999999997 7.679166666666696...
611004,611004,POLYGON ((-76.51249999999996 7.587500000000028...
611005,611005,(POLYGON ((-76.17638888888887 9.37500000000002...


In [20]:
gdf2 = gdf.copy()
gdf2["type"] = gdf2.geometry.geom_type
gdfPolygon = gdf2.loc[gdf2["type"]=="Polygon"]
gdfMultiPolygon = gdf2.loc[gdf2["type"]=="MultiPolygon"]
gdfPolygon2 = gdfPolygon.copy()
gdfMultiPolygon2 = gdfMultiPolygon.copy()

In [21]:
gdfPolygon2['geom'] = gdfPolygon['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
gdfMultiPolygon2['geom'] = gdfMultiPolygon['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))

In [22]:
gdfPolygon2.drop("geometry",1, inplace=True)
gdfMultiPolygon2.drop("geometry",1, inplace=True)

In [23]:
gdfPolygon2.drop("type",1, inplace=True)
gdfMultiPolygon2.drop("type",1, inplace=True)

In [24]:
gdfPolygon2.head()

,pfaf_id2,geom
PFAF_ID,,
611002,611002,POLYGON ((-77.00416666666663 5.770833333333362...
611003,611003,POLYGON ((-76.88749999999997 7.679166666666696...
611004,611004,POLYGON ((-76.51249999999996 7.587500000000028...
611006,611006,"POLYGON ((-76.0208333333333 7.32083333333336, ..."
611008,611008,POLYGON ((-75.18333333333331 10.53750000000002...


In [25]:
tableNamePolygon = TABLE_NAME+"polygon"
tableNameMultiPolygon = TABLE_NAME+"multipolygon"
tableNameGeometries = TABLE_NAME+"geometries"
tableNameAttributes = TABLE_NAME+"attributes"
tableNameOut = TABLE_NAME

In [26]:
gdfPolygon2.to_sql(tableNamePolygon, engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('POLYGON', srid= 4326)})

In [27]:
gdfMultiPolygon2.to_sql(tableNameMultiPolygon, engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid= 4326)})

In [28]:
df = pd.read_csv(os.path.join(EC2_INPUT_PATH,INPUT_FILENAME+".csv"))

In [29]:
df = df.set_index("PFAF_ID", drop=False)

In [30]:
df.columns = map(str.lower, df.columns)

In [31]:
df.to_sql(tableNameAttributes,engine,if_exists='replace', index=False)

### Outer Join

We now have three tables: Polygons, Multipolygons and Attributes. We will perform some operations and perform an outer join.   
Convert polygons to multipolygon and make valid

In [32]:
sql = "ALTER TABLE %s ALTER COLUMN geom type geometry(MultiPolygon, 4326) using ST_Multi(geom);" %(tableNamePolygon)
result = connection.execute(sql)

In [33]:
sql = "CREATE TABLE %s AS (SELECT * FROM %s UNION SELECT * FROM %s);" %(tableNameGeometries, tableNamePolygon,tableNameMultiPolygon)
result = connection.execute(sql)

In [34]:
sql = "update %s set geom = st_makevalid(geom);" %(tableNameGeometries)
result = connection.execute(sql)

In [35]:
sql = "CREATE TABLE %s AS SELECT * FROM %s l FULL OUTER JOIN %s r ON l.pfaf_id2 = r.pfaf_id;" %(tableNameOut,tableNameGeometries,tableNameAttributes)
result = connection.execute(sql)

In [36]:
sql = "DROP TABLE %s,%s,%s,%s" %(tableNamePolygon,tableNameMultiPolygon,tableNameAttributes,tableNameGeometries)
result = connection.execute(sql)

### Testing

In [40]:
sql = "select * from %s" %(tableNameOut)

In [42]:
gdfFromSQL =gpd.GeoDataFrame.from_postgis(sql,connection,geom_col='geom' ).set_index("pfaf_id", drop=False)

In [43]:
gdfFromSQL.head()

,pfaf_id2,geom,pfaf_id,hybas_id2,unnamed: 0,hybas_id,next_down,next_sink,main_bas,dist_sink,...,upstream_hybas_ids,upstream_pfaf_ids,downstream_hybas_ids,downstream_pfaf_ids,next_sink_pfaf,basin_hybas_ids,basin_pfaf_ids,sub_name,maj_name,faoid_copy
pfaf_id,,,,,,,,,,,,,,,,,,,,,
111019,111019,(POLYGON ((35.17777777777781 24.64583333333336...,111019,1060001090,6683,1060001090,0,1060001090,1060001090,0.0,...,[],[],[],[],111019.0,[1060001090],[111019],['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
111084,111084,(POLYGON ((38.56250000000002 15.88750000000003...,111084,1060550700,6701,1060550700,1060525050,1060002760,1060002760,217.4,...,[],[],"[1060525050, 1060002760]","[111083, 111081]",111081.0,"[1060525050, 1060002760, 1060550700]","[111083, 111081, 111084]",['Nahr al Qash'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190318']
111087,111087,(POLYGON ((38.22916666666669 16.00833333333337...,111087,1060581710,6703,1060581710,1060550940,1060002760,1060002760,326.1,...,"[1060606710, 1060606390]","[111088, 111089]","[1060550940, 1060525050, 1060002760]","[111085, 111083, 111081]",111081.0,"[1060550940, 1060525050, 1060002760, 106060671...","[111085, 111083, 111081, 111088, 111089, 111087]",['Nahr al Qash'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190318']
112012,112012,(POLYGON ((47.55833333333336 8.212500000000016...,112012,1060965550,9475,1060965550,1060040270,1060040270,1060040270,0.2,...,[],[],[1060040270],[112011],112011.0,"[1060040270, 1060965550]","[112011, 112012]",['Ogaden'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0193491']
112046,112046,(POLYGON ((46.65000000000002 4.733333333333354...,112046,1061028620,9580,1061028620,1061038600,1060040390,1060040390,88.7,...,[],[],"[1061038600, 1061043750, 1060040390]","[112045, 112043, 112041]",112041.0,"[1061038600, 1061043750, 1060040390, 1061028620]","[112045, 112043, 112041, 112046]",['Ogaden'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0193491']


In [44]:
gdfFromSQL.shape

(16399, 28)

In [37]:
connection.close()

In [38]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:04:44.669692
